In [18]:
import os
import gzip
import json
import pandas as pd
import numpy as np
from collections import defaultdict
from joblib import Parallel, delayed
from tqdm import tqdm
# from tqdm_joblib import tqdm_joblib
import pandas as pd
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentIntensityAnalyzer

import netwulf as nw
import networkx as nx
from networkx.readwrite import json_graph

import community as community_louvain
import matplotlib.pyplot as plt



In [19]:
with open("network.json", "r") as f:
    graph_data = json.load(f)

# Convert the data back into a NetworkX graph object
Graph = json_graph.node_link_graph(graph_data)

In [20]:
num_nodes = Graph.number_of_nodes()
num_edges = Graph.number_of_edges()

#Solve for p using formula:
Lmax = num_nodes * (num_nodes - 1) / 2  
p = num_edges / Lmax  
print(f"Calculated probability p: {p}")

#Find k using formula
average_degree = (2 * num_edges) / num_nodes
print(f"Average degree: {average_degree}")

Calculated probability p: 0.00559690467448642
Average degree: 119.83532598542872


In [21]:
critical_point = 1/num_nodes
print(critical_point)
p > critical_point

4.670278348589576e-05


True

In [22]:
np.log(num_nodes)

9.971706791513272

In [23]:
#list of connected components:
connected_components = list(nx.connected_components(Graph))
#Finding the biggest set, from the list:
largest_component = max(connected_components, key=len)
giant_component = Graph.subgraph(largest_component)

import random
#Algortihm that approximates the shortest path lengths:
def sample_shortest_paths(graph, num_samples):
    nodes = list(graph.nodes())
    total_path_length = 0
    count = 0
    
    for _ in range(num_samples):
        node1, node2 = random.sample(nodes, 2)
        try:
            path_length = nx.shortest_path_length(graph, source=node1, target=node2)
            total_path_length += path_length
            count += 1
        except nx.NetworkXNoPath:
            continue  

    if count > 0:
        return total_path_length / count
    else:
        return float('inf') 

In [24]:
approx_average_path = sample_shortest_paths(giant_component, 10000)
print("Approximated average shortest path", approx_average_path)

Approximated average shortest path 2.6923


In [25]:
def average_dist_approximation(graph):
    N = graph.number_of_nodes()
    avg_degree = sum(dict(graph.degree()).values()) / N
    avg_shortest_path = np.log(N) / np.log(avg_degree)
    return avg_shortest_path

approx_shortest_path_real = average_dist_approximation(giant_component)

print(f"Approx distance in the Soc Sci Network: {approx_shortest_path_real}")

real_clustering = nx.average_clustering(Graph)
print(f"Global Clustering Coefficient of Real Network: {real_clustering}")

Approx distance in the Soc Sci Network: 2.0810850444799396
Global Clustering Coefficient of Real Network: 0.26209674484653656


In [32]:
partition = community_louvain.best_partition(Graph,random_state=100)

In [33]:
communities = defaultdict(list)
for node, comm_id in partition.items():
    communities[comm_id].append(node)

for comm_id, members in communities.items():
    print(f"Community {comm_id}: {members}")

Community 0: ['B0002J58ME', 'B001F76OKC', 'B00147FVKW', 'B0042FDCMW', 'B00B5LIE4C', 'B003X2O79W', 'B00BSB9NDK', 'B006IRQU5S', 'B001OSC4FG', 'B00612FR1I', 'B003ULW71Q', 'B000U0C9UE', 'B00012FNBI', 'B000BR4UQ2', 'B000I2JCB8', 'B00012FNB8', 'B003AFPWI2', 'B07P67V245', 'B0089PCXAS', 'B0009WFFPO', 'B00R8HWMEU', 'B00RC1YRMC', 'B00KE77JJ4', 'B005BX3O0O', 'B00049QPII', 'B00FRPK4GI', 'B00HUAGZKW', 'B00247XURC', 'B00KASQQTQ', 'B000RL21OA', 'B003HC9JJ6', 'B00429C1QQ', 'B004VB5464', 'B08MV1RJXG', 'B08M2M6WK2', 'B07K7SSSPM', 'B07KGJX6N7', 'B07KN9TM7K', 'B07KNB6D5F', 'B07G3KRYZK', 'B07DS8M656', 'B076ZS4LLX', 'B0765925D8', 'B00B2TWDWG', 'B00AZ2N906', 'B00IIOLQXU', 'B00QX5FAQA', 'B01GKKI38Y', 'B01BKTG026', 'B0044CM7YC', 'B0001ZJQ72', 'B00PRABXFO', 'B000059H6M', 'B0009WIY4S', 'B00FXCIS90', 'B00OZOIIYM', 'B00E8A0PNK', 'B0061S2T6S', 'B00R0HUEWK', 'B07T2GN9W2', 'B07DB1LKTH', 'B01M6ATZSG', 'B06Y12RK1V', 'B0012YN35M', 'B00MU1YEWW', 'B00007D00L', 'B00467182C', 'B00W85TAHE', 'B005GYXNT0', 'B000WC38CS', 'B0000

In [34]:
df = pd.read_csv("data/Merged_Reviews_and_Metadata.csv")

C:\Users\mia\AppData\Local\Temp\ipykernel_12516\1340379041.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/Merged_Reviews_and_Metadata.csv")


In [35]:
df.columns

Index(['rating', 'title_x', 'text', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'main_category', 'title_y',
       'average_rating', 'rating_number', 'description', 'price'],
      dtype='object')

In [36]:
community_titles = {}
asin_to_title = dict(zip(df['parent_asin'], df['title_y']))

for comm_id, members in communities.items():
    # Skip single communities
    if len(members) < 2:
        continue

    titles = []
    for asin in members:
        if asin in asin_to_title:
            title = asin_to_title[asin]
            # Skip if the title is NaN
            if pd.isna(title):
                continue
            titles.append(title)

    # Only add the community if it has valid titles
    if titles:
        community_titles[comm_id] = titles

# Print out the movie titles for each community
for comm_id, titles in community_titles.items(): 
    print(f"Community {comm_id}:")
    for title in titles:
        print(f"  > {title}")
    print()

Community 0:
  > 10 Minute Solution: Pilates
  > Rodney Yee's Yoga for Beginners (Packaging May Vary)
  > Lots & Lots of Trucks Volume 1 - Hard Workin Trucks
  > Jane Fonda: Prime Time – Fit & Strong
  > Beachbody Shaun T's Rockin' Body DVD Workout, Dance Workout DVDs, Exercise Videos, Dancing Fitness Guide for Beginners, Seniors, Easy To Follow, Low Impact, 5 Workouts Included
  > Total Immersion Swimming: Perpetual Motion Freestyle in Ten Lessons
  > Kundalini Yoga: Green Energy of the Heart - All-In-One Workout (ALL LEVELS) by Ana Brett & Ravi Singh
  > Jillian Michaels Kickbox FastFix
  > Tricycle: Pick Your Ride Cycle with Mindy Mylrea
  > Cathe Friedrich Low Impact Cycle Max Indoor Cycling DVD - Use Cathe low impact Indoor cycling workouts for lower body sculpting, fat burning, weight loss and aerobic conditioning
  > Bob Harper: Pure Burn Super Strength
  > Viniyoga Therapy for the Low Back, Sacrum & Hips with Gary Kraftsow
  > Yoga Total Tune-UP
  > AM/PM YOGA - For Beginners &